In [1]:
#!/usr/bin/env python
import os
from glob import glob
import re
import pandas as pd

path = os.path.join(os.getcwd(), 'PITCH/PreprocData')
out_dir = os.path.join(os.getcwd(), 'PITCH/Bids')
beh = glob(os.path.join(path, '[0-9][0-9]/[CEa-z]*/[Preost]*/Flanker/run[1-2]/beh/*.txt'))


# rough draft
# Optionally match 'ALL'
pattern = re.compile(r"^.*PreprocData/([0-9]{2})/([CEa-z]*)/([Preost]*)/Flanker/run([1-2])/beh/.*?_(ALL)?.*$")
beh_dict = {}
for filename in beh:
    res = re.search(pattern, filename)
    # we don't want to collect 'ALL'
    if 'ALL' not in res.groups():
        dict_key = '_'.join(res.groups()[:-1])
        if dict_key in beh_dict:
            beh_dict[dict_key].append(filename)
        else:
            beh_dict[dict_key] = [filename]


cond_pattern = re.compile(r"^.*beh/.*?([NCIa-z]*)_([ICa-z]*)_[A-Z]_[RTDa-z]*.txt$")
cond_dict = {}
for sub_stype_ses_run, fnames in beh_dict.items():
    for f in fnames:
        find_cond = re.search(cond_pattern, f)
        cond_corr = '_'.join(find_cond.groups())
        if sub_stype_ses_run in cond_dict:
            if cond_corr in cond_dict[sub_stype_ses_run]:
                cond_dict[sub_stype_ses_run][cond_corr].append(f)
            else:
                cond_dict[sub_stype_ses_run][cond_corr] = [f]
        else:
            cond_dict[sub_stype_ses_run] = {cond_corr: [f]}


In [6]:
def non_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0
headers = ['onset', 'duration', 'response_time', 'correct', 'trial_type']

for sub_stype_ses_run, fdict in cond_dict.items():
    run_df = pd.DataFrame(columns=headers)
    # make the output dataframe for this subject
    for cond_corr, fnames in fdict.items():
        fname_dict = {'Duration': None, 'RT': None}
        cond, corr = cond_corr.split('_')
        for fname in fnames: #has RT and Duration file
            print(fname)
            if 'Duration' in fname:
                col_names = ['onset', 'duration', 'extra1', 'extra2']
                key = 'Duration'
            elif 'RT' in fname:
                col_names = ['onset', 'response_time', 'extra1', 'extra2']
                key = 'RT'
            else:
                print('something is wrong for {fname}'.format(fname=fname))
            if non_zero_file(fname): 
            # load fname into pandas dataframe
                df_fname = pd.read_csv(fname, sep='       ', header=None)
                df_fname.columns = col_names
                df_fname.drop(labels=['extra1', 'extra2'], axis=1, inplace=True)
                fname_dict[key] = df_fname
                new_cols = ['trial_type', 'correct']
                cond_list = [cond for _ in range(len(df_fname))]
                if corr == 'Correct':
                    int_corr = 1
                elif corr == 'Incorrect':
                    int_corr = 0
                else:
                    print('{fcorr} is neither correct nor incorrect'.format(fcorr=corr))
                corr_list = [int_corr for _ in range(len(df_fname))]
                # Add corr_list and cond_list to df_fname
                for lst, coln in zip([cond_list, corr_list], new_cols):
                    se = pd.Series(lst)
                    df_fname[coln] = se.values
        if fname_dict['Duration'] is not None and fname_dict['RT'] is not None:
            df_cond_corr = pd.merge(fname_dict['Duration'], fname_dict['RT'], on='onset')
            # Append the resulting df from the above operation to run_df
            run_df = run_df.append(df_cond_corr)
    
    # write out run_df to disk (as a tsv)
                
        # X combine information from cond_corr (e.g. 'Neutral_Incorrect')
        # X should have both Duration and RT files loaded as DataFrames
        # X append the information to run_df
    # Name and find the place to save the tsv in the BIDS appropiate manner
    # ^^^ Don't use regex, we already have the relevant information from
    # sub_stype_ses_run to save the file in the correct place.
    # "sub-0{sub}/ses-{stype}{ses}/func/sub-0{sub}_ses-{stype}{ses}_task-flanker_run-0{run}_bold.nii.gz".format(your settings here)

# items to learn about:
# 1. format strings: https://www.youtube.com/watch?v=vTX3IwquFkc
# 2. pandas operations:
#    - read_csv
#    - append, concat
#    - to_csv

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run2/beh/P05E1BOL_Incongruent_Correct_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run2/beh/P05E1BOL_Incongruent_Correct_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run2/beh/P05E1BOL_Incongruent_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run2/beh/P05E1BOL_Incongruent_Incorrect_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run2/beh/P05E1BOL_Neutral_Correct_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run2/beh/P05E1BOL_Neutral_Correct_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run2/beh/P05E1BOL_Congruent_Correct_B_Duration.txt
/home/tre

/home/trevor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run1/beh/P05E1BOL_Incongruent_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run1/beh/P05E1BOL_Incongruent_Correct_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run1/beh/P05E1BOL_Neutral_Incorrect_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run1/beh/P05E1BOL_Neutral_Incorrect_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run1/beh/P05E1BOL_Congruent_Incorrect_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run1/beh/P05E1BOL_Congruent_Incorrect_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/05/Exercise/Pre/Flanker/run1/beh/P05E1BOL_Incongruent_Incorrect_A_RT.txt
/home/trevo

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/01/Control/Post/Flanker/run2/beh/P01C2BOL_Neutral_Incorrect_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/01/Control/Post/Flanker/run2/beh/P01C2BOL_Neutral_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/01/Control/Post/Flanker/run2/beh/P01C2BOL_Incongruent_Incorrect_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/01/Control/Post/Flanker/run2/beh/P01C2BOL_Incongruent_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/01/Control/Post/Flanker/run2/beh/P01C2BOL_Congruent_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/01/Control/Post/Flanker/run2/beh/P01C2BOL_Congruent_Incorrect_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/01/Control/Post/Flanker/run1/beh/P01C2BOL_Neutral_Incorrect_A_RT.txt
/home/trevo

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/44/Control/Pre/Flanker/run2/beh/P44C1BOL_Neutral_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/44/Control/Pre/Flanker/run2/beh/P44C1BOL_Incongruent_Correct_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/44/Control/Pre/Flanker/run1/beh/P44C1BOL_Congruent_Incorrect_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/44/Control/Pre/Flanker/run1/beh/P44C1BOL_Incongruent_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/44/Control/Pre/Flanker/run1/beh/P44C1BOL_Neutral_Incorrect_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/44/Control/Pre/Flanker/run1/beh/P44C1BOL_Neutral_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/44/Control/Pre/Flanker/run1/beh/P44C1BOL_Congruent_Correct_A_Duration.txt
/h

...

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/09/Control/Post/Flanker/run2/beh/P09C2BOL_Neutral_Correct_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/09/Control/Post/Flanker/run2/beh/P09C2BOL_Neutral_Correct_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/09/Control/Post/Flanker/run2/beh/P09C2BOL_Incongruent_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/09/Control/Post/Flanker/run2/beh/P09C2BOL_Incongruent_Incorrect_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/09/Control/Post/Flanker/run2/beh/P09C2BOL_Congruent_Correct_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/09/Control/Post/Flanker/run2/beh/P09C2BOL_Congruent_Correct_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/09/Control/Post/Flanker/run1/beh/P09C2BOL_Incongruent_Correct_A_Duration.txt
/home/trevo

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/38/Control/Post/Flanker/run1/beh/P38C2BOL_Incongruent_Incorrect_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/38/Control/Post/Flanker/run1/beh/P38C2BOL_Incongruent_Incorrect_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/38/Exercise/Pre/Flanker/run2/beh/P38E1BOL_Congruent_Correct_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/38/Exercise/Pre/Flanker/run2/beh/P38E1BOL_Congruent_Correct_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/38/Exercise/Pre/Flanker/run2/beh/P38E1BOL_Neutral_Incorrect_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/38/Exercise/Pre/Flanker/run2/beh/P38E1BOL_Neutral_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/38/Exercise/Pre/Flanker/run2/beh/P38E1BOL_Incongruent_Incorrect_B_RT.txt
/home/trevo

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/10/Exercise/Pre/Flanker/run2/beh/P10E1BOL_Congruent_Incorrect_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/10/Exercise/Pre/Flanker/run2/beh/P10E1BOL_Congruent_Incorrect_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/10/Exercise/Pre/Flanker/run2/beh/P10E1BOL_Neutral_Correct_B_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/10/Exercise/Pre/Flanker/run2/beh/P10E1BOL_Neutral_Correct_B_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/10/Exercise/Pre/Flanker/run1/beh/P10E1BOL_Neutral_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/10/Exercise/Pre/Flanker/run1/beh/P10E1BOL_Neutral_Correct_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/10/Exercise/Pre/Flanker/run1/beh/P10E1BOL_Incongruent_Correct_A_RT.txt
/home/trevor/PitchToBIDS/

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/28/Exercise/Post/Flanker/run1/beh/P28E2BOL_Congruent_Correct_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/28/Exercise/Post/Flanker/run1/beh/P28E2BOL_Congruent_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/28/Exercise/Post/Flanker/run1/beh/P28E2BOL_Neutral_Correct_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/28/Exercise/Post/Flanker/run1/beh/P28E2BOL_Neutral_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/28/Exercise/Post/Flanker/run1/beh/P28E2BOL_Neutral_Incorrect_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/28/Exercise/Post/Flanker/run1/beh/P28E2BOL_Neutral_Incorrect_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/28/Exercise/Post/Flanker/run1/beh/P28E2BOL_Congruent_Incorrect_A_Duration.txt
/home/trevor

/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/08/Exercise/Post/Flanker/run1/beh/P08E2BOL_Congruent_Incorrect_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/08/Exercise/Post/Flanker/run1/beh/P08E2BOL_Congruent_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/08/Exercise/Post/Flanker/run1/beh/P08E2BOL_Congruent_Correct_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/08/Exercise/Post/Flanker/run1/beh/P08E2BOL_Neutral_Correct_A_Duration.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/08/Exercise/Post/Flanker/run1/beh/P08E2BOL_Neutral_Correct_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/08/Exercise/Post/Flanker/run1/beh/P08E2BOL_Incongruent_Correct_A_RT.txt
/home/trevor/PitchToBIDS/2PitchToBids/PitchToBids/PITCH/PreprocData/08/Exercise/Post/Flanker/run1/beh/P08E2BOL_Incongruent_Correct_A_Duration.txt
/home/trevor/P

In [7]:
run_df

,correct,correct_x,correct_y,duration,onset,response_time,trial_type,trial_type_x,trial_type_y
0,NaN,0.0,0.0,0.51662,263.119510,1.51664,NaN,Neutral,Neutral
1,NaN,0.0,0.0,0.51662,265.102690,0.69033,NaN,Neutral,Neutral
0,NaN,0.0,0.0,0.51662,8.023201,1.36053,NaN,Congruent,Congruent
1,NaN,0.0,0.0,0.51662,155.511430,1.51664,NaN,Congruent,Congruent
0,NaN,1.0,1.0,0.51661,29.971458,0.73170,NaN,Congruent,Congruent
1,NaN,1.0,1.0,0.51661,45.920186,0.76655,NaN,Congruent,Congruent
2,NaN,1.0,1.0,0.51662,49.886529,0.45610,NaN,Congruent,Congruent
3,NaN,1.0,1.0,0.51662,61.868901,0.72940,NaN,Congruent,Congruent
4,NaN,1.0,1.0,0.51662,73.784621,1.03738,NaN,Congruent,Congruent
5,NaN,1.0,1.0,0.51662,103.682231,1.33105,NaN,Congruent,Congruent


In [ ]:
df_fname

In [ ]:
beh_dict['01_Control_Post_1']

In [ ]:
cond_dict

In [ ]:
len(df_fname) * 'string'

In [ ]:
['string' for _ in range(len(df_fname))]

In [ ]:
range(3)

In [ ]:
for x in range(3):
    print(x)

In [ ]:
y = range(3)

In [ ]:
y.count

In [ ]:
def factorial(x):
    if x == 1:
        return 1
    return x * factorial(x-1)

In [ ]:
factorial(5)